In [139]:
from utils.base_data_handler import BaseDataHandler
import pandas as pd

In [140]:
handler = BaseDataHandler('2025-12-05/datasets/train_new.csv')

handler.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    891 non-null    int64  
 1   PassengerId   891 non-null    int64  
 2   Survived      891 non-null    int64  
 3   Pclass        891 non-null    int64  
 4   Name          891 non-null    object 
 5   Sex           891 non-null    object 
 6   Age           891 non-null    float64
 7   SibSp         891 non-null    int64  
 8   Parch         891 non-null    int64  
 9   Ticket        891 non-null    object 
 10  Fare          891 non-null    float64
 11  Cabin         891 non-null    object 
 12  Embarked      891 non-null    object 
 13  Age Category  891 non-null    object 
dtypes: float64(2), int64(6), object(6)
memory usage: 97.6+ KB


In [141]:
handler.try_clean_column_names()
handler.df.nunique()

unnamed_0       891
passenger_id    891
survived          2
pclass            3
name            891
sex               2
age              89
sib_sp            7
parch             7
ticket          681
fare            248
cabin           148
embarked          4
age_category      3
dtype: int64

In [142]:
handler.df.drop(['unnamed_0', 'name'], inplace=True, axis=1)
handler.df.nunique()

passenger_id    891
survived          2
pclass            3
sex               2
age              89
sib_sp            7
parch             7
ticket          681
fare            248
cabin           148
embarked          4
age_category      3
dtype: int64

In [143]:
handler.df = pd.get_dummies(handler.df, columns=['sex', 'embarked', 'age_category'], drop_first=True)
handler.df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   passenger_id         891 non-null    int64  
 1   survived             891 non-null    int64  
 2   pclass               891 non-null    int64  
 3   age                  891 non-null    float64
 4   sib_sp               891 non-null    int64  
 5   parch                891 non-null    int64  
 6   ticket               891 non-null    object 
 7   fare                 891 non-null    float64
 8   cabin                891 non-null    object 
 9   sex_male             891 non-null    bool   
 10  embarked_C           891 non-null    bool   
 11  embarked_Q           891 non-null    bool   
 12  embarked_S           891 non-null    bool   
 13  age_category_Senior  891 non-null    bool   
 14  age_category_Young   891 non-null    bool   
dtypes: bool(6), float64(2), int64(5), object

In [144]:
handler.try_add_col('cabin_zone', lambda row: row['cabin'][0])
handler.df = pd.get_dummies(handler.df, columns=['cabin_zone'], drop_first=True)
handler.try_add_col('cabin_price_proxy', lambda r: r['fare']/(r['sib_sp'] + r['parch'] + 1))
handler.df.drop(columns=['ticket', 'cabin'], axis=1, inplace=True)
handler.df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   passenger_id         891 non-null    int64  
 1   survived             891 non-null    int64  
 2   pclass               891 non-null    int64  
 3   age                  891 non-null    float64
 4   sib_sp               891 non-null    int64  
 5   parch                891 non-null    int64  
 6   fare                 891 non-null    float64
 7   sex_male             891 non-null    bool   
 8   embarked_C           891 non-null    bool   
 9   embarked_Q           891 non-null    bool   
 10  embarked_S           891 non-null    bool   
 11  age_category_Senior  891 non-null    bool   
 12  age_category_Young   891 non-null    bool   
 13  cabin_zone_A         891 non-null    bool   
 14  cabin_zone_B         891 non-null    bool   
 15  cabin_zone_C         891 non-null    boo

In [ ]:
import optuna
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# --- 1. Preparazione Dati ---
X, y = handler.get_training_data('survived')
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

X_train_safe = np.asarray(X_train)
y_train_safe = np.asarray(y_train)
X_test_safe  = np.asarray(X_test)
y_test_safe  = np.asarray(y_test)

# wrap training and test sets in DMatrix (GPU)
dtrain = xgb.DMatrix(X_train_safe, label=y_train_safe) 
dtest = xgb.DMatrix(X_test_safe, label=y_test_safe)


# --- 2. Objective Function ---
def objective(trial):
    params = {
        "objective": "binary:logistic",
        "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        #"tree_method": "hist",
        #"device": "cuda",
        "seed": 42,
        "eval_metric": "error",
    }

    num_round = trial.suggest_int("n_estimators", 100, 1000)

    xgb_cv = xgb.cv(
        params,
        dtrain,
        num_boost_round=num_round,
        early_stopping_rounds=20,
        nfold=5,
        stratified=True,
        seed=42,
        metrics="error",
    )

    return 1.0 - xgb_cv["test-error-mean"].min()

# --- 3. Run Optuna study ---
print("--- Bayesian Optimization ---")
study = optuna.create_study(direction="maximize", study_name="XGBoost_BreastCancer_GPU")
study.optimize(objective, n_trials=50, show_progress_bar=False)

# --- 4. Results ---
print("\n--- Optimization's results ---")
print(f"Best Trial (Trial #{study.best_trial.number})")
print(f"Highest Accuracy (CV): {study.best_value:.4f}")
print("Best Parameters:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# --- 5. Validazione Finale sul Test Set ---
best_params = study.best_params

num_boost_round = best_params.pop('n_estimators')

best_params.update({
    "objective": "binary:logistic",
    "tree_method": "hist",
    "device": "cuda",
    "seed": 42,
    "eval_metric": "error",
})

final_model = xgb.train(best_params, dtrain, num_boost_round=num_boost_round)
y_pred = final_model.predict(dtest)
y_pred_labels = (y_pred).astype(int)

final_acc = accuracy_score(y_test_safe, y_pred_labels)
print(f"\nFinal accuracy on test set: {final_acc:.4f}")

[I 2025-12-05 12:56:52,085] A new study created in memory with name: XGBoost_BreastCancer_GPU


--- Bayesian Optimization ---


[I 2025-12-05 12:56:52,442] Trial 0 finished with value: 0.818851570964247 and parameters: {'learning_rate': 0.018368406907749105, 'max_depth': 8, 'subsample': 0.625724650452452, 'colsample_bytree': 0.5644980551544448, 'reg_alpha': 0.0019097484714106507, 'reg_lambda': 1.4371527262776521e-06, 'n_estimators': 921}. Best is trial 0 with value: 0.818851570964247.
[I 2025-12-05 12:56:52,528] Trial 1 finished with value: 0.8005811090318132 and parameters: {'learning_rate': 0.03360965667970914, 'max_depth': 3, 'subsample': 0.7694990029320425, 'colsample_bytree': 0.8511538357175967, 'reg_alpha': 0.6657507746539776, 'reg_lambda': 1.07686929927097e-06, 'n_estimators': 530}. Best is trial 0 with value: 0.818851570964247.
[I 2025-12-05 12:56:52,590] Trial 2 finished with value: 0.8187924751305033 and parameters: {'learning_rate': 0.2687409565514819, 'max_depth': 6, 'subsample': 0.5357555734885132, 'colsample_bytree': 0.8225845413996755, 'reg_alpha': 0.07079959417297081, 'reg_lambda': 7.85944792400


--- Optimization's results ---
Best Trial (Trial #27)
Highest Accuracy (CV): 0.8314
Best Parameters:
  learning_rate: 0.21530553082283893
  max_depth: 10
  subsample: 0.747742737383061
  colsample_bytree: 0.6791505086621535
  reg_alpha: 0.28780029231383286
  reg_lambda: 1.3346412243442965e-07
  n_estimators: 612


TypeError: expected str, bytes or os.PathLike object, not DataFrame

In [146]:
# --- 6. Visualizzazione Avanzata ---
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice

# A. Storia dell'ottimizzazione
# Vediamo se il modello sta migliorando nel tempo
fig1 = plot_optimization_history(study)
fig1.show()

# B. Importanza degli Iperparametri
# Quale parametro impatta di più l'accuratezza? (Spesso è il Learning Rate)
fig2 = plot_param_importances(study)
fig2.show()

# C. Slice Plot
# Vediamo dove si concentrano i punti migliori per ogni parametro
fig3 = plot_slice(study)
fig3.show()